In [1]:
from SBM_SDE import *
import obs_and_flow_old_hyb as old
import torch
from torch import nn
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [38]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = 0.2 #SDE discretization timestep.
t = 220 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 5e-4
niter = 5800
piter = 500
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
obs_error_scale = 0.25

In [39]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [40]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [41]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object =D.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [42]:
def train_old_flow(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) 
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFLOW(DEVICE, obs_model, STATE_DIM, T, DT, N).to(DEVICE) #For old obs_and_flow_classes_and_functions.py
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e10
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for i in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net(BATCH_SIZE) #For old obs_and_flow_classes_and_functions. #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if i <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if i % 10 == 0:
                    ma_norm_loss = sum(norm_losses[-10:]) / len(norm_losses[-10:])
                    print(f"\nMoving average norm loss at {iter} iterations is: {ma_norm_loss}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), DT, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if i % 10 == 0:             
                    ma_elbo_loss = sum(ELBO_losses[-10:]) / len(ELBO_losses[-10:])
                    print(f"\nMoving average ELBO loss at {iter} iterations is: {ma_elbo_loss}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                neg_ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if i % 100000 == 0 and i > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()
    return net, ELBO_losses, norm_losses

In [ ]:
net, ELBO_losses, norm_losses = train_old_flow(devi, l_r, niter, piter, batch_size, old.SDEFlow, old.ObsModel, old.csv_to_obs_df, 'CON_synthetic_sol_df.csv', obs_error_scale, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/5800 [00:00<?, ?it/s]


Moving average norm loss at <built-in function iter> iterations is: 9000010002.322657. Best norm loss value is: 100023.2265625.

C_PATH mean = tensor([[0.8919, 0.6507, 0.6792],
        [0.8760, 0.6526, 0.6817]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.6648,  1.3519,  0.7198],
         [ 1.0234,  0.6221,  0.8231],
         ...,
         [ 1.1302,  0.6009,  0.4865],
         [ 0.8886,  0.6995,  0.4947],
         [ 0.7331,  0.7060,  0.2196]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.6648,  0.5454,  0.7604],
         [ 0.9840,  0.4832,  0.7798],
         ...,
         [ 0.9269,  0.6506,  0.6702],
         [ 1.4672,  0.5507,  0.5885],
         [ 1.3566,  0.6553,  0.2515]]], grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 10/5800 [00:08<1:23:01,  1.16it/s]


Moving average norm loss at <built-in function iter> iterations is: 95875.37890625. Best norm loss value is: 93507.265625.

C_PATH mean = tensor([[3.5606, 0.1810, 0.6812],
        [3.3543, 0.1860, 0.6832]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.3506,  0.2947,  0.3584],
         [ 0.3175,  0.4369,  0.3896],
         ...,
         [ 1.9945,  0.3302,  0.9112],
         [ 0.2650,  0.4051,  0.7555],
         [ 0.4345,  0.3669,  0.2308]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.5043,  3.6210,  0.4044],
         [ 1.1341,  0.7433,  0.3847],
         ...,
         [ 7.8516,  0.3108,  0.8225],
         [ 3.3433,  0.2864,  0.5546],
         [ 5.6015,  0.3706,  0.2488]]], grad_fn=<CatBackward>)



Train Diffusion:   0%|          | 20/5800 [00:17<1:30:31,  1.06it/s]

...


Train Diffusion:  84%|████████▍ | 4900/5800 [1:17:21<15:32,  1.04s/it]


Moving average ELBO loss at <built-in function iter> iterations is: -13760.4150390625. Best ELBO loss value is: -14674.6826171875.

C_PATH mean = tensor([[40.1497,  0.0909,  0.7109],
        [40.1605,  0.0964,  0.7183]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.4873e+01, 2.4475e-02, 4.3377e-01],
         [4.5242e+01, 6.5836e-04, 4.2081e-01],
         ...,
         [3.6855e+01, 1.2545e-01, 8.0304e-01],
         [3.7220e+01, 6.9638e-02, 8.8788e-01],
         [3.6763e+01, 1.0909e-01, 7.3962e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5467e+01, 6.6130e-02, 1.9125e-01],
         [4.5103e+01, 6.4873e-02, 4.0364e-01],
         ...,
         [3.7600e+01, 4.9230e-02, 4.5730e-01],
         [3.7133e+01, 9.4174e-02, 4.4779e-01],
         [3.7496e+01, 9.7501e-02, 5.8804e-01]]], grad_fn=<CatBackward>)



Train Diffusion:  85%|████████▍ | 4910/5800 [1:17:30<13:30,  1.10it/s]


Moving average ELBO loss at <built-in function iter> iterations is: -13812.33388671875. Best ELBO loss value is: -14674.6826171875.

C_PATH mean = tensor([[40.1536,  0.0930,  0.7163],
        [40.1705,  0.0884,  0.7267]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5150e+01, 7.9690e-02, 3.6770e-01],
         [4.5583e+01, 8.9863e-02, 3.9006e-01],
         ...,
         [3.7729e+01, 1.1051e-01, 5.2933e-01],
         [3.7233e+01, 8.0072e-02, 6.2681e-01],
         [3.7698e+01, 7.8531e-02, 4.8883e-01]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01],
         [4.5788e+01, 3.6681e-02, 2.5101e-01],
         [4.5376e+01, 1.0809e-03, 4.0956e-01],
         ...,
         [3.6798e+01, 6.2746e-02, 7.4563e-01],
         [3.7256e+01, 8.7031e-02, 6.0349e-01],
         [3.6883e+01, 1.2699e-01, 6.7050e-01]]], grad_fn=<CatBackward>)



Train Diffusion:  85%|████████▍ | 4919/5800 [1:17:39<14:01,  1.05it/s]

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

len(ELBO_losses)
len(norm_losses)

